##  To do:
- hh; jobs (office, industrial, retail); housing units (sf, mf)
    - 2019 (average of runs)
    - 2050 (average of runs)

- append to parcel eq table
    - create some pivots

In [56]:
import os
import pandas as pd
import glob

# show all columns
pd.options.display.max_columns = None

In [57]:
outputs = os.path.join(r'.\Outputs')
if not os.path.exists(outputs):
    os.makedirs(outputs)

In [58]:
# Parcel Equivalency Table
eq = pd.read_csv(r".\Inputs\parcel_eq_v5.csv")

# get 2019 and 2050 files
remm_pm_2019_1 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_1\REMMRun\Progression_Metrics\run_48_year_2019_parcel_progression_metrics.csv")
remm_pm_2050_1 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_1\REMMRun\Progression_Metrics\run_48_year_2050_parcel_progression_metrics.csv")

remm_pm_2019_2 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_2\REMMRun\Progression_Metrics\run_48_year_2019_parcel_progression_metrics.csv")
remm_pm_2050_2 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_2\REMMRun\Progression_Metrics\run_48_year_2050_parcel_progression_metrics.csv")

remm_pm_2019_3 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_3\REMMRun\Progression_Metrics\run_48_year_2019_parcel_progression_metrics.csv")
remm_pm_2050_3 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_3\REMMRun\Progression_Metrics\run_48_year_2050_parcel_progression_metrics.csv")

remm_pm_2019_4 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_4\REMMRun\Progression_Metrics\run_47_year_2019_parcel_progression_metrics.csv")
remm_pm_2050_4 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_4\REMMRun\Progression_Metrics\run_47_year_2050_parcel_progression_metrics.csv")

remm_pm_2019_5 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_5\REMMRun\Progression_Metrics\run_47_year_2019_parcel_progression_metrics.csv")
remm_pm_2050_5 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_5\REMMRun\Progression_Metrics\run_47_year_2050_parcel_progression_metrics.csv")

remm_pm_2019_6 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_6\REMMRun\Progression_Metrics\run_46_year_2019_parcel_progression_metrics.csv")
remm_pm_2050_6 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_6\REMMRun\Progression_Metrics\run_46_year_2050_parcel_progression_metrics.csv")

In [59]:
def prepare_df(df, year):
    df = df.set_index('parcel_id')
    df.loc[(df['is_sf']==1), 'sf_units'] = df['residential_units']
    df.loc[(df['is_mf']==1), 'mf_units'] = df['residential_units']
    df['ind_jobs'] = df['jobs_wholesale'] + df['jobs_manuf']
    df['rtl_jobs'] = df['jobs_retail'] + df['jobs_accom_food']
    df['off_jobs'] = df['jobs_office'] + df['jobs_gov_edu'] + df['jobs_health'] + df['jobs_other']

    df = df[['sf_units', 'mf_units', 'ind_jobs', 'rtl_jobs', 'off_jobs']].copy()
    new_col_names = [col + f'_{year}' for col in list(df.columns)]
    df.columns = new_col_names
    return df.fillna(0)

In [60]:
remm_pm_2019 = [remm_pm_2019_1, remm_pm_2019_2, remm_pm_2019_3, remm_pm_2019_4, remm_pm_2019_5, remm_pm_2019_6]
remm_pm_2050 = [remm_pm_2050_1, remm_pm_2050_2, remm_pm_2050_3, remm_pm_2050_4, remm_pm_2050_5, remm_pm_2050_6]

remm_pm_2019_processed = [prepare_df(df, 2019) for df in remm_pm_2019]
remm_pm_2050_processed = [prepare_df(df, 2050) for df in remm_pm_2050]

In [61]:
# stack all 6 runs together
data_stack_2019 = pd.concat(remm_pm_2019_processed)
data_stack_2050 = pd.concat(remm_pm_2050_processed)

# average the 6 runs
results_2019 = data_stack_2019.groupby(data_stack_2019.index).mean().reset_index().round().astype(int)
results_2050 = data_stack_2050.groupby(data_stack_2050.index).mean().reset_index().round().astype(int)

results_2019['residential_units_2019'] = results_2019['sf_units_2019'] + results_2019['mf_units_2019']
results_2019['ttl_jobs_2019'] = results_2019['off_jobs_2019'] + results_2019['rtl_jobs_2019'] + results_2019['ind_jobs_2019']
results_2050['residential_units_2050'] = results_2050['sf_units_2050'] + results_2050['mf_units_2050']
results_2050['ttl_jobs_2050'] = results_2050['off_jobs_2050'] + results_2050['rtl_jobs_2050'] + results_2050['ind_jobs_2050']


In [62]:
# fill na's in parcel eq
eq.fillna({'CENTER_NAME':'Non-Center', 'CENTER_TYPE':'Non-Center', 'CITY_AREA':'Unincorporated'}, inplace=True)

In [63]:
# export
results = eq.merge(results_2019, on='parcel_id', how='left').merge(results_2050, on='parcel_id', how='left')
results.to_csv(os.path.join(outputs, 'SE_by_PARCEL.csv'), index = False)

In [64]:
# eq.columns

In [65]:
results.columns

Index(['parcel_id', 'TAZID_832', 'TAZID_900', 'CENTER_NAME', 'CENTER_TYPE',
       'CITY_AREA', 'COUNTY_NAME', 'COUNTY_FIPS', 'MPO', 'INFILL1990',
       'parcel_acres', 'sf_units_2019', 'mf_units_2019', 'ind_jobs_2019',
       'rtl_jobs_2019', 'off_jobs_2019', 'residential_units_2019',
       'ttl_jobs_2019', 'sf_units_2050', 'mf_units_2050', 'ind_jobs_2050',
       'rtl_jobs_2050', 'off_jobs_2050', 'residential_units_2050',
       'ttl_jobs_2050'],
      dtype='object')

In [66]:
# pivot to different geographies
data_columns = ['parcel_acres', 'residential_units_2019', 'sf_units_2019',
       'mf_units_2019', 'ind_jobs_2019', 'rtl_jobs_2019', 'off_jobs_2019', 'ttl_jobs_2019',
       'residential_units_2050', 'sf_units_2050', 'mf_units_2050',
       'ind_jobs_2050', 'rtl_jobs_2050', 'off_jobs_2050','ttl_jobs_2050']

# aggregations
results.groupby('CITY_AREA')[data_columns].sum().reset_index().to_csv(os.path.join(outputs, 'SE_by_CITY_AREA.csv'), index = False)
results.groupby('COUNTY_NAME')[data_columns].sum().reset_index().to_csv(os.path.join(outputs, 'SE_by_COUNTY.csv'), index = False)
results.groupby('CENTER_NAME')[data_columns].sum().reset_index().to_csv(os.path.join(outputs, 'SE_by_CENTER_NAME.csv'), index = False)
results.groupby('CENTER_TYPE')[data_columns].sum().reset_index().to_csv(os.path.join(outputs, 'SE_by_CENTER_TYPE.csv'), index = False)
results.groupby('INFILL1990')[data_columns].sum().reset_index().to_csv(os.path.join(outputs, 'SE_by_INFILL1990.csv'), index = False)
results.groupby('MPO')[data_columns].sum().reset_index().to_csv(os.path.join(outputs, 'SE_by_MPO.csv'), index = False)

# pivots
results.pivot_table(index='CENTER_TYPE', columns='COUNTY_NAME', values=data_columns,  aggfunc='sum').fillna(0).reset_index().to_csv(os.path.join(outputs, 'SE_by_CENTER_TYPE_by_COUNTY.csv'), index = False)
results.pivot_table(index='COUNTY_NAME', columns='INFILL1990', values=data_columns,  aggfunc='sum').fillna(0).reset_index().to_csv(os.path.join(outputs, 'SE_by_INFILL1990_by_COUNTY.csv'), index = False)